# Исследование надёжности заёмщиков

**Цель исследования** – выяснить, влияет ли семейное положение и количество детей клиента на погашение кредита в срок. Результаты исследования будут учтены при построении модели кредитного скоринга.

**Задачи исследования:**
* Проверить, есть ли зависимость между количеством детей и возвратом кредита в срок.
* Посмотреть, есть ли зависимость между зависимость между семейным положением и возвратом кредита в срок.
* Проверить, есть ли зависимость между между уровнем дохода и возвратом кредита в срок.
* Узнать, как разные цели кредита влияют на его возврат в срок.

**Ход исследования:**
1. Обзор данных.
2. Предобработка данных и подготовка их к анализу.
3. Исследовательский анализ данных.
4. Общие выводы.

**Описание данных**

`children` — количество детей в семье\
`days_employed` — общий трудовой стаж в днях\
`dob_years` — возраст клиента в годах\
`education` — уровень образования клиента\
`education_id` — идентификатор уровня образования\
`family_status` — семейное положение\
`family_status_id` — идентификатор семейного положения\
`gender` — пол клиента\
`income_type` — тип занятости\
`debt` — имел ли задолженность по возврату кредитов\
`total_income` — ежемесячный доход\
`purpose` — цель получения кредита

# 1. Обзор данных

In [1]:
# импортируем библиотеки необходимые для анализа
import pandas as pd

In [2]:
data = pd.read_csv('/Users/polinamokretsova/data.csv')

In [3]:
# выведем первые строки датасета 
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
# посмотрим общую информацию о датасете
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Мы ознакомились с данными: 
* Загрузили датасет и посмотрели общую информацию о данных.
* Обнаружили, что столбец days_employed содержит отрицательные значения. Данные в этом столбце необходимо исправить на положительные.
* Столбец total_income имеет вещественный тип данных. Для удобства изменим его на целочисленный.
* Столбец education содержит дубликаты - одна и та же категория образования написана по-разному.

Приступим к предообраюотке данных и подготовим их к анализу.

## 2. Предобработка данных

**Проверим данные на пропуски**

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Мы обнаружили одинаковое количество пропусков в двух столбцах: days_employed и total_income.

**Обработаем пропуски в столбце total_income**

Столбец **total_income** содержит информацию о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому логично будет заполнить пропуски в этом столбце медианным значением по каждому типу из столбца income_type. 

In [6]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

**Проверим данные на аномальные значения**

In [7]:
data['days_employed'].unique()

array([-8437.67302776, -4024.80375385, -5623.42261023, ...,
       -2113.3468877 , -3112.4817052 , -1984.50758853])

Столбец с количеством дней трудового стажа содержит отрицательные значения. Такие значения не реальны и, вероятно, появились по какой-то ошибке. Обработаем значения в этом столбце: заменим все отрицательные значения положительными.

In [8]:
data['days_employed'] = data['days_employed'].abs()

**Для каждого типа занятости выведем медианное значение трудового стажа days_employed в днях**

In [9]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов людей (безработные и пенсионеры) получились аномально большие значения. Мы не сможем достоверно изменить значения в столбце, поэтому оставим данные, как есть. Это будет не критично, так как не отразится на результате исследования.

**Посмотрим на уникальные значения столбца children**

In [10]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Как мы видим, в наших данных есть два аномальных значения: -1 и 20. Исправим это, очистив их из нашего датасета,

In [11]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [12]:
# проверим, что очистка прошла успешно
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

**Обработаем пропуски в столбце days_employed**

Заполним пропуски в столбце days_employed медианными значениями по каждого типа занятости income_type.

In [13]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [14]:
# убедимся, что пропуски заполнены
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Пропуски заполнены успешно

**Изменим тип данных в столбце total_income на целочисленный**

In [15]:
data['total_income'] = data['total_income'].astype(int)

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21402 non-null  int64  
 1   days_employed     21402 non-null  float64
 2   dob_years         21402 non-null  int64  
 3   education         21402 non-null  object 
 4   education_id      21402 non-null  int64  
 5   family_status     21402 non-null  object 
 6   family_status_id  21402 non-null  int64  
 7   gender            21402 non-null  object 
 8   income_type       21402 non-null  object 
 9   debt              21402 non-null  int64  
 10  total_income      21402 non-null  int64  
 11  purpose           21402 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB


**Проверим данные на явные дубликаты**

In [17]:
data.duplicated().sum()

54

Мы обнаружили 54 явных дубликата, удалим их из наших данных.

In [18]:
data = data.drop_duplicates()

In [19]:
# проверим на дубликаты повторно
data.duplicated().sum()

0

**Проверим данные на неявные дубликаты**

В столбце education одни и те же категории образования написаны по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру и проверим на дубликаты еще раз.

In [20]:
data['education'] = data['education'].str.lower()

In [21]:
data.duplicated().sum()

17

Мы обнаружили 54 явных дубликата, удалим их из наших данных.

In [22]:
data = data.drop_duplicates()
# проверим на дубликаты повторно
data.duplicated().sum()

0

**Категоризируем данные**

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [23]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [24]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [25]:
# проверим, что столбец добавился
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C


**Посмотрим перечень уникальных целей взятия кредита из столбца purpose**

In [26]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Одни и те же цели кредита могут быть написаны по-разному. Приведем к единым категориям.

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [27]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [28]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

In [29]:
# проверим на уникальные значения столбец purpose_category
data['purpose_category'].unique()

array(['операции с недвижимостью', 'операции с автомобилем',
       'получение образования', 'проведение свадьбы'], dtype=object)

In [30]:
# проверим финально данные на дубликаты
data.duplicated().sum()

0

Мы провели предобработку данных:
* Проверили данные на пропуски и обработали их.
* Проверили данные на явные и неявные дубликаты. Очистили данные от повторяющихся строк.
* Проверили данные на аномалии, избавились от выбросов в столбце children. Столбец days_employed мы оставили без изменений, так как мы не сможем достоверно изменить значения. Это будет не критично, так как не отразится на результате исследования.  
* Категоризировали данные. 

Перейдем к исследованию.

## 3. Исследовательский анализ данных

Создадим функцию, которая поможет нам расчитать долю задолженностей по разным категориям людей.

На вход функция будет принимать столбец, по которому мы хотим определить долю задолжников. И узнать есть ли зависимость между определенным признаком и возвратном предита в срок. 

In [31]:
# вводим функцию
def create_pivot(column):
    return data.pivot_table(index=[column], values='debt', aggfunc='mean').sort_values(by='debt', ascending=False)

**Проверим, есть ли зависимость между количеством детей и возвратом кредита в срок**

In [32]:
create_pivot('children')

,debt
children,
4,0.097561
2,0.094542
1,0.092346
3,0.081818
0,0.075438
5,0.000000


**Вывод:** 

- Бездетные семьи чаще платят кредиты точно в срок, в отличие от семей с бóльшим количеством детей. 
- Семьи с тремя детьми возвращают кредит точно в срок чаще, чем семьи с одним и двумя детьми.
- Данных о семьях с пятью детьми недостаточно для оценки этой категории. Сейчас мы можем сделать вывод, что эти семьи реже остальных берут кредиты.

Наших данных недостаточно, чтобы дать однозначный ответ о зависимости возврата кредита точно в срок от количества детей. Для этого может потребоваться выборка больше.

**Проверим, есть ли зависимость между зависимость между семейным положением и возвратом кредита в срок**

In [33]:
create_pivot('family_status')

,debt
family_status,
Не женат / не замужем,0.097639
гражданский брак,0.093130
женат / замужем,0.075606
в разводе,0.070648
вдовец / вдова,0.066246


**Вывод:** 
- Заёмщики, имеющие семейное положение "не женат/не замужем" и "гражданский брак", реже всего платят кредит ровно в срок.
- Средние показатели у заёмщиков, имеющие семейное положение "женат/замужем" и "в разводе". При этом у заёмщиков, находящихся в разводе, всё же показатели возврата кредита точно в срок лучше.
- Чаще точно в срок возвращают кредит вдова/вдовец.

**Проверим, есть ли зависимость между между уровнем дохода и возвратом кредита в срок**

In [34]:
create_pivot('total_income_category')

,debt
total_income_category,
E,0.090909
C,0.084982
A,0.080000
B,0.070602
D,0.060172


**Вывод:** 
- По нашим данным мы не можем установить точную зависимость между уровнем дохода и возвратом кредита в срок. Данные по самому высокому уровню дохода и самому низкому нерепрезентативны относительно остальных, их меньше всего.

**Узнаем, как разные цели кредита влияют на его возврат в срок**

In [35]:
create_pivot('purpose_category')

,debt
purpose_category,
операции с автомобилем,0.093480
получение образования,0.092528
проведение свадьбы,0.079118
операции с недвижимостью,0.072551


**Вывод:** 
- Точно в срок возвращают кредит за недвижимость. Почти такие же показатели у возврата кредитов за проведение свадьбы.
- Больше задолженностей у заёмщиков за кредиты на автомобили и образование.

## 4. Общий вывод

Мы провели исследование, чтобы выяснить, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Какие вы выводы в ходе анализа мы получили:
- Заёмщики, имеющие семейное положение "не женат/не замужем" и "гражданский брак", реже всего платят кредит ровно в срок. В то время, как вдова/вдовец возвращают кредит вовремя.
- Бездетные семьи чаще платят кредиты точно в срок, в отличие от семей с бóльшим количеством детей. При этом семьи с тремя детьми возвращают кредит точно в срок чаще, чем семьи с одним и двумя детьми.